# Deep Q-Leaning on the Lunar Lander V2 Problem (OpenAI Gym)

In [ ]:
!pip install gym[box2d]==0.17

import gym
import numpy as np

In [ ]:
"""
Constants for DQN Agent
"""

epsilon = 1 # amount of exploration
gamma = .99 # temporal discount
learning_rate = 0.001 # alpha
tau = 0.01 # update rate of target network

memory_buffer_size= 500000 # number of experiences possibly stored at once
batch_size = 64 # number of samples in each batch
n_actions = 4
state_dim = 8
n_experiments = 5
n_episodes_dqn = 500

from enum import Enum
class DecayType(Enum):
    EXPONENTIAL = 0
    LINEAR = 1

epsilon_decay_type = DecayType.LINEAR

lin_epsilon_decay_param = 0.002 # constant subtraction
exp_epsilon_decay_param = 0.99 # constant multiplication
min_eps = 0.01 # final epsilon value


In [ ]:
class MemoryBuffer:
    """
    Stores experiences (state, action, reward, new_state, done) for i.i.d. assumption during learning
    """ 
    def __init__(self):
        self.states = np.zeros((memory_buffer_size, 8), np.float64)
        self.actions = np.zeros(memory_buffer_size, np.intc)
        self.rewards = np.zeros(memory_buffer_size, np.float64)
        self.new_states = np.zeros((memory_buffer_size, 8), np.float64)
        self.dones = np.zeros(memory_buffer_size, np.bool_)

        self.head = 0

    def write(self, state, action, reward, new_state, done):
        index = self.head % memory_buffer_size

        self.states[index] = state
        self.actions[index] = action
        self.rewards[index] = reward
        self.new_states[index] = new_state
        self.dones[index] = done

        self.head += 1

    def sample(self):
        sample = np.random.choice(min(self.head, memory_buffer_size), batch_size)

        return (self.states[sample],
                self.actions[sample],
                self.rewards[sample],
                self.new_states[sample],
                self.dones[sample])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.activations import relu, linear
from keras import Sequential

class DQN(Sequential):
    def __init__(self):
        super().__init__()
        self.add(keras.layers.Dense(256, input_dim=state_dim, activation=relu))
        self.add(keras.layers.Dense(256, activation=relu))
        self.add(keras.layers.Dense(n_actions, activation=linear))
        self.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=learning_rate ))

In [ ]:
class DQNAgent:
    """
    Agent that plays and learns.
    It has access to the memory buffer and stores the DQN and DQN target.
    """ 
    def __init__(self, buffer):
        self.epsilon = epsilon
        self.buffer = buffer
        self.dqn = DQN()
        self.dqn_target = DQN()

    def get_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(n_actions)
        state = np.array([state])
        qs = self.dqn(state)
        return np.argmax(qs)

    def update_target(self):
        new_weights = []
        target_weights = self.dqn_target.get_weights()
        main_weights = self.dqn.get_weights()
        for target_weight, main_weight in zip(target_weights, main_weights):
            new_weights.append((1 - tau) * target_weight + tau * main_weight)
        self.dqn_target.set_weights(new_weights)

    def update(self):
        states, actions, rewards, new_states, dones = self.buffer.sample()
        # we change y only in the (s, a) positions where experiences are available
        y = np.copy(self.dqn(states))
        next_qs = self.dqn_target(new_states)
        max_next_qs = np.amax(next_qs, axis=1)

        for idx in range(batch_size):
            # times done to avoid learning over final experiences
            y[idx, actions[idx]] = rewards[idx] + (1 - dones[idx]) * gamma * max_next_qs[idx]

        self.dqn.train_on_batch(states, y)
        self.update_target()

        if self.epsilon > min_eps:
            if epsilon_decay_type == DecayType.LINEAR:
                self.epsilon -= lin_epsilon_decay_param
            elif epsilon_decay_type == DecayType.EXPONENTIAL:
                self.epsilon *= exp_epsilon_decay_param
            else:
                print("please choose decay type")
                exit()

In [ ]:
"""
Main experiment loop for DQN Agent
""" 

env = gym.make("LunarLander-v2")

cum_rewards, timesteps, epsilons = [], [], []

for experiment in range(n_experiments):

  buffer = MemoryBuffer()
  agent = DQNAgent(buffer)

  cum_rewards_experiment = []
  timesteps_experiment = []

  for episode in range(n_episodes_dqn):
    state = env.reset()
    t = 0
    cum_reward_episode = 0
    while True:
      prev_state = state
      action = agent.get_action(state)
      state, reward, done, _ = env.step(action)
      cum_reward_episode += reward
      buffer.write(prev_state, action, reward, state, done)
      agent.update()
      if done:
        print(f"Episode {episode+1} - {t+1} timesteps, cum_reward = {cum_reward_episode} \n")
        timesteps_experiment.append(t+1)
        cum_rewards_experiment.append(cum_reward_episode)
        if experiment == 0:
          epsilons.append(agent.epsilon)
        break
      t += 1

  cum_rewards.append(cum_rewards_experiment)
  timesteps.append(timesteps_experiment)
  print(f"Experiment {experiment+1} finished.\n")

env.close()